### *IPCC SR15 scenario assessment*

<img style="float: right; height: 80px; padding-left: 20px;" src="../_static/IIASA_logo.png">
<img style="float: right; height: 80px;" src="../_static/IAMC_logo.jpg">

# Analysis of short-lived non-CO2 emissions

This notebook plots emissions of CH4, F-gases, BC and SO2 as shown in **Figure 2.7**
of the IPCC's _"Special Report on Global Warming of 1.5°C"_.

The scenario data used in this analysis can be accessed and downloaded at [https://data.ene.iiasa.ac.at/iamc-1.5c-explorer](https://data.ene.iiasa.ac.at/iamc-1.5c-explorer).

## Load `pyam` package and other dependencies

In [ ]:
import pandas as pd
import numpy as np
import warnings
import io
import itertools
import yaml
import math
import matplotlib.pyplot as plt
plt.style.use('style_sr15.mplstyle')
%matplotlib inline
import pyam

from utils import boxplot_by_cat

## Import scenario data, categorization and specifications files

The metadata file with scenario categorisation and quantitative indicators can be downloaded at [https://data.ene.iiasa.ac.at/iamc-1.5c-explorer](https://data.ene.iiasa.ac.at/iamc-1.5c-explorer).  
Alternatively, it can be re-created using the notebook `sr15_2.0_categories_indicators`.

The last cell of this section loads and assigns a number of auxiliary lists as defined in the categorization notebook.

In [ ]:
sr1p5 = pyam.IamDataFrame(data='../data/iamc15_scenario_data_world_r2.0.xlsx')

In [ ]:
sr1p5.load_meta('sr15_metadata_indicators.xlsx')

In [ ]:
with open("sr15_specs.yaml", 'r') as stream:
    specs = yaml.load(stream, Loader=yaml.FullLoader)

rc = pyam.run_control()
for item in specs.pop('run_control').items():
    rc.update({item[0]: item[1]})
cats = specs.pop('cats')
all_cats = specs.pop('all_cats')
subcats = specs.pop('subcats')
all_subcats = specs.pop('all_subcats')
plotting_args = specs.pop('plotting_args')
marker= specs.pop('marker')

## Downselect scenario ensemble to categories of interest for this assessment

In [ ]:
years = [2010, 2030, 2050]

In [ ]:
cats.remove('Above 2C')

In [ ]:
sr1p5.meta.rename(columns={'Kyoto-GHG|2010 (SAR)': 'kyoto_ghg_2010'}, inplace=True)

In [ ]:
df = sr1p5.filter(kyoto_ghg_2010='in range', category=cats, year=years)

## Set specifications for filter and plotting

In [ ]:
save_name = 'output/fig2.7{}.{}'

In [ ]:
filter_args = dict(df=sr1p5, category=cats, marker=None, join_meta=True)

In [ ]:
compare_years = [2030, 2050]
base_year = 2010

In [ ]:
def plotting_args(name, filetype='png'):
    return {'categories': cats, 'column': 'category', 'years': years,
            'add_marker': marker,
            'save': save_name.format(name, filetype)}

In [ ]:
data = []

## Plot different emissions pathways by category

In [ ]:
ch4 = df.filter(variable='Emissions|CH4').timeseries()

In [ ]:
name = 'ch4'
label = 'Global CH4 emissions'
unit = 'MtCH4/yr'
_data = pyam.filter_by_meta(ch4, **filter_args)
fig = boxplot_by_cat(_data, ylabel='{} ({})'.format(label, unit),
                     **plotting_args('a_{}'.format(name)))
_data['species'] = name
data.append(_data)

In [ ]:
fgases = df.filter(variable='Emissions|F-Gases').timeseries()

In [ ]:
name = 'f-gases'
label = 'Global F-gas emissions'
unit = 'GtCO2e/yr'

_data = pyam.filter_by_meta(fgases, **filter_args)
fig = boxplot_by_cat(_data, ylabel='{} ({})'.format(label, unit),
                     **plotting_args('b_{}'.format(name)), legend=False)
_data['species'] = name
data.append(_data)

In [ ]:
bc = df.filter(variable='Emissions|BC').timeseries()

In [ ]:
name = 'bc'
label = 'Global BC emissions'
unit = 'MtBC'

_data = pyam.filter_by_meta(bc, **filter_args)
fig = boxplot_by_cat(_data, ylabel='{} ({})'.format(label, unit),
                     **plotting_args('c_{}'.format(name)), legend=False)
_data['species'] = name
data.append(_data)

In [ ]:
so2 = df.filter(variable='Emissions|Sulfur').timeseries()

In [ ]:
name = 'so2'
label = 'Global SO2 emissions'
unit = 'MtSO2'
_data = pyam.filter_by_meta(so2, **filter_args)
fig = boxplot_by_cat(_data, ylabel='{} ({})'.format(label, unit),
                     **plotting_args('d_{}'.format(name)), legend=False)
_data['species'] = name
data.append(_data)

## Export timeseries data to `xlsx`

In [ ]:
data = pd.concat(data).set_index(['species', 'category', 'marker'], append=True)

In [ ]:
data.head()

In [ ]:
data.reset_index().to_excel('output/fig2.7_data_table.xlsx')